In [1]:
import pandas as pd
import numpy as np 
import matplotlib as mpl
import seaborn as sns

In [2]:
FEATURE = "Labor_Force_Gender_Ratio"
df = pd.read_csv("..\..\datasets\Original\%s.csv" % FEATURE)
df

,Series Name,Series Code,Country Name,Country Code,2012 [YR2012],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Ratio of female to male labor force participat...,SL.TLF.CACT.FM.ZS,Afghanistan,AFG,20.7785921101157,27.1038412375653,28.9361754413037,29.4739178697398,30.0098803224511,24.6858778957205
1,Ratio of female to male labor force participat...,SL.TLF.CACT.FM.ZS,Albania,ALB,74.8163273282632,76.6604881917464,74.5183571677263,76.1208647546987,77.829119439077,75.8574481545003
2,Ratio of female to male labor force participat...,SL.TLF.CACT.FM.ZS,Algeria,DZA,23.3033867432138,25.4773705466379,25.2636177431848,25.0532818615054,24.8778600511553,24.1241414205571
3,Ratio of female to male labor force participat...,SL.TLF.CACT.FM.ZS,American Samoa,ASM,..,..,..,..,..,..
4,Ratio of female to male labor force participat...,SL.TLF.CACT.FM.ZS,Andorra,AND,..,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...
217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,Data from database: Gender Statistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Skeleton dataframe

In [3]:
columns = ["Country Code"]
df.drop_duplicates(subset=columns, keep="first")
df_new = df[columns[0]].unique()
df_new = pd.DataFrame(df_new, columns=columns)
df.rename(columns={"Country Code": "ISO3_Code"}, inplace=True)
df_new.rename(columns={"Country Code": "ISO3_Code"}, inplace=True)
df_new.shape

(218, 1)

In [4]:
df_code = pd.read_csv("../../datasets/Original/AreaCode_ISO3Code_conversion.csv")
df_code = df_code[["Country", "ISO3 Code", "M49 Code"]]
df_new = df_new.join(df_code.set_index("ISO3 Code"), on="ISO3_Code", validate="1:1")
df_new

,ISO3_Code,Country,M49 Code
0,AFG,Afghanistan,4.0
1,ALB,Albania,8.0
2,DZA,Algeria,12.0
3,ASM,American Samoa,16.0
4,AND,Andorra,20.0
...,...,...,...
213,PSE,Palestine,275.0
214,YEM,Yemen,887.0
215,ZMB,Zambia,894.0
216,ZWE,Zimbabwe,716.0


In [5]:
df_new.rename(columns={"ISO3 Code": "ISO3_Code", "Country": "Country_Name", "M49 Code": "M49_Code"}, inplace=True)
df_new.head()

,ISO3_Code,Country_Name,M49_Code
0,AFG,Afghanistan,4.0
1,ALB,Albania,8.0
2,DZA,Algeria,12.0
3,ASM,American Samoa,16.0
4,AND,Andorra,20.0


In [6]:
df_new[df_new["Country_Name"].isnull()]

,ISO3_Code,Country_Name,M49_Code
39,CHI,NaN,NaN
105,XKX,NaN,NaN
217,NaN,NaN,NaN


In [7]:
# drop the null rows cause it's the legend
df_new.dropna(inplace=True)
df_new

,ISO3_Code,Country_Name,M49_Code
0,AFG,Afghanistan,4.0
1,ALB,Albania,8.0
2,DZA,Algeria,12.0
3,ASM,American Samoa,16.0
4,AND,Andorra,20.0
...,...,...,...
212,VIR,United States Virgin Islands,850.0
213,PSE,Palestine,275.0
214,YEM,Yemen,887.0
215,ZMB,Zambia,894.0


## Unemployment
Unemployment (% of labor force)

In [8]:
df_tmp = pd.concat([df.loc[:, ["ISO3_Code"]], df.iloc[:, -5:]], axis=1)
df_new = df_new.merge(df_tmp, on="ISO3_Code", how="inner")
columns = df.columns[-5:].values
names_dict = {}
for i, column in enumerate(columns):
    names_dict[column] = "%s_%s" % (FEATURE, column[:4])
df_new.rename(columns=names_dict, inplace=True)
df_new

,ISO3_Code,Country_Name,M49_Code,Labor_Force_Gender_Ratio_2016,Labor_Force_Gender_Ratio_2017,Labor_Force_Gender_Ratio_2018,Labor_Force_Gender_Ratio_2019,Labor_Force_Gender_Ratio_2020
0,AFG,Afghanistan,4.0,27.1038412375653,28.9361754413037,29.4739178697398,30.0098803224511,24.6858778957205
1,ALB,Albania,8.0,76.6604881917464,74.5183571677263,76.1208647546987,77.829119439077,75.8574481545003
2,DZA,Algeria,12.0,25.4773705466379,25.2636177431848,25.0532818615054,24.8778600511553,24.1241414205571
3,ASM,American Samoa,16.0,..,..,..,..,..
4,AND,Andorra,20.0,..,..,..,..,..
...,...,...,...,...,...,...,...,...
210,VIR,United States Virgin Islands,850.0,71.8052360545949,70.8705491998924,70.3064581330838,69.5917422099498,68.974480394609
211,PSE,Palestine,275.0,24.5415613975824,24.7721340528117,25.1662343501475,25.8342023707959,24.7541140919029
212,YEM,Yemen,887.0,9.0315791406404,8.99949794310724,8.99992611462765,9.00792495806297,9.00915545979032
213,ZMB,Zambia,894.0,87.6535251841707,88.0900635480689,88.5855310716293,89.0886797382114,89.0880281074896


In [9]:
# add a column for average over the years
columns = df_new.columns[-5:]
df_cols = df_new[columns].applymap(lambda x: float(x) if x not in ["", None, ".."] else None)
df_new = pd.concat([df_new.iloc[:, :3], df_cols], axis=1)
df_new["%s_Avg" % FEATURE] = df_new[columns].mean(axis=1)
df_new.head()

,ISO3_Code,Country_Name,M49_Code,Labor_Force_Gender_Ratio_2016,Labor_Force_Gender_Ratio_2017,Labor_Force_Gender_Ratio_2018,Labor_Force_Gender_Ratio_2019,Labor_Force_Gender_Ratio_2020,Labor_Force_Gender_Ratio_Avg
0,AFG,Afghanistan,4.0,27.103841,28.936175,29.473918,30.009880,24.685878,28.041939
1,ALB,Albania,8.0,76.660488,74.518357,76.120865,77.829119,75.857448,76.197256
2,DZA,Algeria,12.0,25.477371,25.263618,25.053282,24.877860,24.124141,24.959254
3,ASM,American Samoa,16.0,NaN,NaN,NaN,NaN,NaN,NaN
4,AND,Andorra,20.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# prettify the float
#df_new = df_new[df_new.columns[-6:]].applymap(na_action='ignore', func=lambda x: ".6f" % x)
with open("../../datasets/Processed/%s_Processed.csv" % FEATURE, "w", encoding="utf8") as f:
    df_new.to_csv(f, lineterminator="\n", index=False)

## Stats

In [11]:
df_new.isnull().sum()

ISO3_Code                         0
Country_Name                      0
M49_Code                          0
Labor_Force_Gender_Ratio_2016    29
Labor_Force_Gender_Ratio_2017    29
Labor_Force_Gender_Ratio_2018    29
Labor_Force_Gender_Ratio_2019    29
Labor_Force_Gender_Ratio_2020    29
Labor_Force_Gender_Ratio_Avg     29
dtype: int64